In [242]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [171]:
train_data = pd.read_csv('../input/titanic/train.csv')
test_data = pd.read_csv('../input/titanic/test.csv')
PassengerId = test_data['PassengerId'].values

In [172]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [173]:
train_data[['Pclass', 'Survived']].groupby('Pclass').mean().sort_values(by='Survived', ascending=False)

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [174]:
train_data[['SibSp', 'Survived']].groupby('SibSp').mean().sort_values(by='Survived', ascending=False)

,Survived
SibSp,
1,0.535885
2,0.464286
0,0.345395
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [175]:
train_data[['Embarked', 'Survived']].groupby('Embarked').mean().sort_values(by='Survived', ascending=False)

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [176]:
full_data = pd.concat([train_data, test_data])

In [177]:
full_data[full_data['Embarked'].isnull() == True]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [178]:
most_freq_embarked_stn = full_data['Embarked'].dropna().mode()[0]
most_freq_embarked_stn

'S'

In [179]:
full_data['Embarked'].fillna(most_freq_embarked_stn, inplace=True)

In [180]:
full_data['Embarked'] = full_data['Embarked'].map({'C':0, 'Q':1, 'S':2}).astype('int')
full_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,2


In [181]:
full_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,2


In [182]:
full_data.describe(include=['O'])

,Name,Sex,Ticket,Cabin
count,1309,1309,1309,295
unique,1307,2,929,186
top,"Kelly, Mr. James",male,CA. 2343,C23 C25 C27
freq,2,843,11,6


In [183]:
full_data['Sex'] = full_data['Sex'].map({'male':0, 'female':1}).astype('int')

In [184]:
full_data.drop('Cabin', axis=1, inplace=True)

In [185]:
full_data.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,0
2,3,1.0,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,2


In [186]:
full_data[['Pclass', 'Sex', 'Embarked']] = full_data[[ 'Pclass', 'Sex', 'Embarked']].astype('category')

In [187]:
full_data.dtypes

PassengerId       int64
Survived        float64
Pclass         category
Name             object
Sex            category
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Embarked       category
dtype: object

In [188]:
full_data['Title'] = full_data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)


In [189]:
full_data['Title'] = full_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
full_data['Title'] = full_data['Title'].replace('Mlle', 'Miss')
full_data['Title'] = full_data['Title'].replace('Ms', 'Miss')
full_data['Title'] = full_data['Title'].replace('Mme', 'Mrs')

In [190]:
full_data.drop('Name', axis=1, inplace=True)

In [191]:
full_data.drop('Ticket', axis=1, inplace=True)

In [192]:
full_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0.0,3,0,22.0,1,0,7.2500,2,Mr
1,2,1.0,1,1,38.0,1,0,71.2833,0,Mrs
2,3,1.0,3,1,26.0,0,0,7.9250,2,Miss
3,4,1.0,1,1,35.0,1,0,53.1000,2,Mrs
4,5,0.0,3,0,35.0,0,0,8.0500,2,Mr


In [195]:
age_mean = round(full_data['Age'].dropna().mean(), 0)
age_mean

30.0

In [198]:
full_data['Age'].fillna(age_mean, inplace=True)

In [201]:
full_data['Age_Cat'] = pd.qcut(full_data['Age'], 4)

In [202]:
full_data['Age_Cat'] = full_data['Age_Cat'].astype('category')
full_data['Age_Cat'] = full_data['Age_Cat'].cat.codes

In [205]:
full_data['Fare_Cat'] = pd.qcut(full_data['Fare'], 4)

In [206]:
full_data['Fare_Cat'] = full_data['Fare_Cat'].astype('category')
full_data['Fare_Cat'] = full_data['Fare_Cat'].cat.codes

In [208]:
full_data.drop(['Fare','Age'], axis=1, inplace=True)

In [210]:
full_data['Family_Size'] = full_data['SibSp'] + full_data['Parch'] + 1

In [212]:
full_data['IsAlone'] = 0

for dataset in full_data:
    full_data.loc[full_data['Family_Size'] == 1, 'IsAlone'] = 1

In [216]:
full_data.drop(['SibSp','Parch'], axis=1, inplace=True)

KeyError: "['SibSp' 'Parch'] not found in axis"

In [217]:
full_data['Survived'].replace(np.nan, -1, inplace=True)

In [220]:
full_data.head()

,PassengerId,Survived,Pclass,Sex,Embarked,Title,Age_Cat,Fare_Cat,Family_Size,IsAlone
0,1,0.0,3,0,2,Mr,0,0,2,0
1,2,1.0,1,1,0,Mrs,3,3,2,0
2,3,1.0,3,1,2,Miss,1,1,1,1
3,4,1.0,1,1,2,Mrs,2,3,2,0
4,5,0.0,3,0,2,Mr,2,1,1,1


In [221]:
full_data['Survived'].value_counts()

 0.0    549
-1.0    418
 1.0    342
Name: Survived, dtype: int64

In [251]:
full_data.head()

,PassengerId,Survived,Pclass,Sex,Embarked,Title,Age_Cat,Fare_Cat,Family_Size,IsAlone
0,1,0.0,3,0,2,1,0,0,2,0
1,2,1.0,1,1,0,3,3,3,2,0
2,3,1.0,3,1,2,2,1,1,1,1
3,4,1.0,1,1,2,3,2,3,2,0
4,5,0.0,3,0,2,1,2,1,1,1


In [252]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     1309 non-null   float64
 2   Pclass       1309 non-null   int64  
 3   Sex          1309 non-null   int64  
 4   Embarked     1309 non-null   int64  
 5   Title        1309 non-null   int64  
 6   Age_Cat      1309 non-null   int8   
 7   Fare_Cat     1309 non-null   int8   
 8   Family_Size  1309 non-null   int64  
 9   IsAlone      1309 non-null   int64  
dtypes: float64(1), int64(7), int8(2)
memory usage: 94.6 KB


In [247]:
full_data[['Pclass','Sex','Embarked']] = full_data[['Pclass','Sex','Embarked']].astype('int64')

In [250]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
full_data['Title'] = full_data['Title'].map(title_mapping)

In [253]:
train_data = full_data[full_data['Survived']!=-1]

In [254]:
test_data = full_data[full_data['Survived']== -1]

In [255]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Survived     418 non-null    float64
 2   Pclass       418 non-null    int64  
 3   Sex          418 non-null    int64  
 4   Embarked     418 non-null    int64  
 5   Title        418 non-null    int64  
 6   Age_Cat      418 non-null    int8   
 7   Fare_Cat     418 non-null    int8   
 8   Family_Size  418 non-null    int64  
 9   IsAlone      418 non-null    int64  
dtypes: float64(1), int64(7), int8(2)
memory usage: 30.2 KB


In [256]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Embarked     891 non-null    int64  
 5   Title        891 non-null    int64  
 6   Age_Cat      891 non-null    int8   
 7   Fare_Cat     891 non-null    int8   
 8   Family_Size  891 non-null    int64  
 9   IsAlone      891 non-null    int64  
dtypes: float64(1), int64(7), int8(2)
memory usage: 64.4 KB


In [257]:
test_data.drop('Survived', axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [258]:
test_data.drop('PassengerId', axis=1, inplace=True)
train_data.drop('PassengerId', axis=1, inplace=True)

In [261]:
X_train = train_data.drop(['Survived'], axis=1).values
Y_train = train_data['Survived'].ravel()
X_test = test_data.values

In [263]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

81.48

In [265]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

84.51

In [266]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

78.9

In [267]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

78.34

In [268]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


81.03

In [269]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

78.34

In [270]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

88.66

In [264]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

83.28

In [262]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

88.66

In [271]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,88.66
8,Decision Tree,88.66
1,KNN,84.51
0,Support Vector Machines,83.28
2,Logistic Regression,81.48
7,Linear SVC,81.03
4,Naive Bayes,78.90
5,Perceptron,78.34
6,Stochastic Gradient Decent,78.34


In [ ]:
prediction = pd.DataFrame({'PassengerId':PassengerId, 'Survived': Y_pred })